In [1]:
#! /usr/bin/env python
"""
Compute debris thickness through sub-debris and temperature inversion methods
"""
import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
from scipy import ndimage
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.stats import median_absolute_deviation
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib


import debrisglobal.globaldebris_input as debris_prms
from debrisglobal.glacfeat import GlacFeat, create_glacfeat
from meltcurves import melt_fromdebris_func
from meltcurves import debris_frommelt_func
from spc_split_lists import split_list


debug=False

In [2]:
#Function to generate a 3-panel plot for input arrays
def plot_array(dem, clim=None, titles=None, cmap='inferno', label=None, overlay=None, fn=None, close_fig=True):
    fig, ax = plt.subplots(1,1, sharex=True, sharey=True, figsize=(10,5))
    alpha = 1.0
    #Gray background
    ax.set_facecolor('0.5')
    #Force aspect ratio to match images
    ax.set(aspect='equal')
    #Turn off axes labels/ticks
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if titles is not None:
        ax.set_title(titles[0])
    #Plot background shaded relief map
    if overlay is not None:
        alpha = 0.7
        ax.imshow(overlay, cmap='gray', clim=(1,255))
    #Plot each array
    im_list = [ax.imshow(dem, clim=clim, cmap=cmap, alpha=alpha)]
    fig.tight_layout()
    fig.colorbar(im_list[0], label=label, extend='both', shrink=0.5)
    if fn is not None:
        fig.savefig(fn, bbox_inches='tight', pad_inches=0, dpi=150)
    if close_fig:
        plt.close(fig)
        
        
def maskedarray_gt(data, value):
    """ Greater than operation on masked array to avoid warning errors """
    data = np.nan_to_num(data,0)
    data[data > value] = value
    return data


def maskedarray_lt(data, value):
    """ Less than operation on masked array to avoid warning errors """
    data = np.nan_to_num(data,0)
    data[data < value] = value
    return data


def ts_fromdebris_func(h, a, b, c):
    """ estimate surface temperature from debris thickness (h is debris thickness, a and k are coefficients) 
        Hill Equation"""
    return a * h**c / (b**c + h**c)


def debris_fromts_func(ts, a, b, c, hd_max=debris_prms.hd_max):
    """ estimate debris thickness from surface temperature (ts is surface temperature, a and k are coefficients) 
        Hill Equation"""
    # If temperature exceeds maximum of function cause NaN value
    max_value = ts_fromdebris_func(50, a, b, c)
    if ts.size == 1:
        if ts > max_value:
            ts = max_value
        if ts < 0:
            ts = 0
    else:
        ts[ts > a] = max_value
        ts[ts < 0] = 0
    # Debris thickness
    hd = (ts * b**c / (a - ts))**(1/c)
    return hd
        
    
def debris_fromts_maskedarray(ts_raw, a, b, c):
    """ Apply debris_fromts_func to masked array
        includes a mask of maximum values, since Michaelis-Mentin Equation has natural maximum 
    Parameters
    ----------
    ts_raw : np.ma.array
        masked array of the unmodified surface temperature
    Returns
    -------
    hd : np.ma.array 
        masked array of the debris thickness (m)
    """
    hd = debris_fromts_func(ts_raw.data, a, b, c)
    return hd

In [3]:
# ===== DETERMINE ALL GLACIERS WITH AND WITHOUT OBSERVATIONS =====
hdopt_prms_fp = debris_prms.output_fp + 'hd_opt_prms/' + debris_prms.roi + '/'

# Glaciers optimized
glac_tsopt_fns = []
rgiid_list_tsopt = []
for i in os.listdir(hdopt_prms_fp):
    if i.endswith('_hdopt_prms.csv'):
        region = int(i.split('.')[0])
        if region in debris_prms.roi_rgidict[debris_prms.roi]:    
            rgiid_list_tsopt.append(i.split('_')[0])            
            glac_tsopt_fns.append(i)
glac_tsopt_fns = sorted(glac_tsopt_fns)
rgiid_list_tsopt = sorted(rgiid_list_tsopt)

main_glac_rgi_tsopt = debris_prms.selectglaciersrgitable(rgiid_list_tsopt)

# All debris-covered glaciers
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_rgiid = sorted([x.split('-')[1] for x in dc_shp.RGIId])
main_glac_rgi_all = debris_prms.selectglaciersrgitable(glac_no=dc_rgiid)

# Merge with debris cover stats
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_shp = dc_shp.sort_values(by=['RGIId'])
dc_shp.reset_index(inplace=True, drop=True)

# Add debris stats to area
dc_areaperc_dict = dict(zip(dc_shp.RGIId.values,dc_shp['DC_Area__1'].values))
dc_area_dict = dict(zip(dc_shp.RGIId.values,dc_shp['DC_Area_v2'].values))

main_glac_rgi_tsopt['DC_Area_%'] = main_glac_rgi_tsopt.RGIId.map(dc_areaperc_dict).fillna(0)
main_glac_rgi_all['DC_Area_%'] = main_glac_rgi_all.RGIId.map(dc_areaperc_dict).fillna(0)
main_glac_rgi_tsopt['DC_Area_v2'] = main_glac_rgi_tsopt['Area'] * main_glac_rgi_tsopt['DC_Area_%'] / 100
main_glac_rgi_all['DC_Area_v2'] = main_glac_rgi_all['Area'] * main_glac_rgi_all['DC_Area_%'] / 100

# Glaciers lacking optimization
rgiids_missing = set(main_glac_rgi_all.rgino_str.values) - set(main_glac_rgi_tsopt.rgino_str.values)
rgiids_missing = sorted(rgiids_missing)
main_glac_rgi_missing = debris_prms.selectglaciersrgitable(rgiids_missing)
main_glac_rgi_missing

73 glaciers in region 3 are included in this model run: ['00527', '00718', '00923', '00928', '00937', '00945', '00987', '01005', '01192', '01210', '01341', '01354', '01476', '01479', '01603', '01616', '01669', '01705', '01739', '01756', '01885', '01889', '01891', '01907', '02017', '02143', '02147', '02149', '02168', '02184', '02260', '02268', '02272', '02273', '02282', '02295', '02296', '02298', '02300', '02304', '02320', '02352', '02355', '02365', '02384', '02385', '02397', '02410', '02418', '02422'] and more
This study is focusing on 73 glaciers in region [3]
2330 glaciers in region 3 are included in this model run: ['00093', '00094', '00095', '00096', '00097', '00098', '00099', '00100', '00102', '00103', '00104', '00105', '00106', '00107', '00108', '00109', '00110', '00113', '00114', '00115', '00116', '00117', '00118', '00120', '00123', '00124', '00125', '00126', '00128', '00129', '00130', '00131', '00132', '00133', '00134', '00135', '00136', '00137', '00139', '00140', '00141', '001

,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,Slope,Aspect,Lmax,Form,TermType,Surging,RefDate,glacno,rgino_str,RGIId_float
GlacNo,,,,,,,,,,,,,,,,,,,,
0,92,RGI60-03.00093,-72.332200,80.836100,3,3,267.736,740,1819,1545,8.2,46,27392,1,0,0,20019999,93,03.00093,3.00093
1,93,RGI60-03.00094,-72.338500,80.920900,3,3,25.570,716,1555,1198,6.9,28,11300,1,0,0,20019999,94,03.00094,3.00094
2,94,RGI60-03.00095,-72.672400,80.916700,3,3,87.814,736,1685,1421,7.9,37,16626,1,0,0,20019999,95,03.00095,3.00095
3,95,RGI60-03.00096,-72.746100,80.983600,3,3,23.206,902,1349,1302,7.9,23,3923,1,0,0,20019999,96,03.00096,3.00096
4,96,RGI60-03.00097,-72.705800,81.019900,3,3,14.714,900,1263,1093,3.8,26,8405,0,0,0,20019999,97,03.00097,3.00097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2252,4539,RGI60-03.04542,-90.390887,76.319507,3,6,14.137,182,550,398,3.0,28,6985,0,0,0,19990705,4542,03.04542,3.04542
2253,4540,RGI60-03.04543,-90.328452,76.304371,3,6,9.574,207,550,398,3.7,33,5029,0,0,0,19990705,4543,03.04543,3.04543
2254,4541,RGI60-03.04544,-90.265386,76.287974,3,6,19.344,103,496,367,3.0,48,6879,0,0,0,19990705,4544,03.04544,3.04544


In [4]:
print('\n', debris_prms.roi + ': calibration includes', main_glac_rgi_tsopt.shape[0], 'glaciers covering',
      str(np.round(main_glac_rgi_tsopt['DC_Area_v2'].sum(),1)), 'km2 (' + 
      str(np.round(main_glac_rgi_tsopt['DC_Area_v2'].sum() / main_glac_rgi_all['DC_Area_v2'].sum() * 100,1)) + 
      '%) of the total debris-covered glacier area\n')     


 03: calibration includes 73 glaciers covering 186.5 km2 (11.4%) of the total debris-covered glacier area



In [5]:
# # print('\nHACK TO EXTRAPOLATE TO CALIBRATED GLACIERS FOR COMPARISON\n')
# rgiids_missing = ['15.02167']
# # # rgiids_missing = ['2.14297']
# # # rgiids_missing = ['11.02472']
# # # rgiids_missing = ['12.01132']
# # # rgiids_missing = ['13.43165']
# # # rgiids_missing = ['15.03473']
# # # rgiids_missing = ['15.04045']
# # # rgiids_missing = ['18.02397']
# # rgiids_missing = ['11.03206']

# main_glac_rgi_missing = debris_prms.selectglaciersrgitable(rgiids_missing)

In [6]:
extrap_uncalibrated_glaciers = True
overwrite_hd = False

hd_fp = debris_prms.hd_fp + 'extrap/'
if not os.path.exists(hd_fp):
    os.makedirs(hd_fp)
    
mf_fp = hd_fp + 'meltfactor/'
if not os.path.exists(mf_fp):
    os.makedirs(mf_fp)
    
fig_extrap = debris_prms.output_fig_fp + debris_prms.roi + '/' + 'extrap/'
if not os.path.exists(fig_extrap):
    os.makedirs(fig_extrap)

if extrap_uncalibrated_glaciers:
    
    # ===== NEAREST GLACIERS WITH DATA =====
    n_glac_nearest = 1000
    if n_glac_nearest > main_glac_rgi_tsopt.shape[0]:
        n_glac_nearest = main_glac_rgi_tsopt.shape[0]

    nearest_dict = {}
    for nglac, glac_idx in enumerate(main_glac_rgi_missing.index.values):
#     for nglac, glac_idx in enumerate([main_glac_rgi_missing.index.values[755]]):
        glac_str = main_glac_rgi_missing.loc[glac_idx, 'rgino_str']
        if glac_idx%500 == 0:
            print(glac_idx, glac_str)
        latlon_dist = (((main_glac_rgi_tsopt['CenLat'].values - main_glac_rgi_missing['CenLat'].values[glac_idx])**2 + 
                        (main_glac_rgi_tsopt['CenLon'].values - main_glac_rgi_missing['CenLon'].values[glac_idx])**2)**0.5)

        latlon_nearidx_list = np.argsort(latlon_dist)[0:n_glac_nearest]
        rgiid_nearest_list = list(main_glac_rgi_tsopt.loc[latlon_nearidx_list,'rgino_str'].values)

        nearest_dict[glac_str] = rgiid_nearest_list

    # Ts filenames
    ts_fns_df = pd.read_csv(debris_prms.ts_fp + debris_prms.ts_fns_fn)
    
    for nglac, glac_idx in enumerate(main_glac_rgi_missing.index.values):
#     for nglac, glac_idx in enumerate(main_glac_rgi_missing.index.values[150:200]):
#     for nglac, glac_idx in enumerate([main_glac_rgi_missing.index.values[0]]):
        glac_str = main_glac_rgi_missing.loc[glac_idx, 'rgino_str']
        rgiid = main_glac_rgi_missing.loc[glac_idx,'RGIId']
        region = glac_str.split('.')[0]

        if int(region) < 10:
            glac_str_noleadzero = str(int(glac_str.split('.')[0])) + '.' + glac_str.split('.')[1]
        else:
            glac_str_noleadzero = glac_str

        # Ts filename
        ts_fn_idx = np.where(ts_fns_df['RGIId'].values == rgiid)[0][0]
        ts_fn = ts_fns_df.loc[ts_fn_idx,'ts_fullfn']
        
        # Hd filename
        hd_fn = debris_prms.hd_fn_sample.replace('XXXX', glac_str_noleadzero).replace('.tif','_extrap.tif')

        # Ice thickness filenames
        thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
        thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
        
        if ((not os.path.exists(hd_fp + hd_fn) or overwrite_hd) and os.path.exists(thick_dir + thick_fn) and 
            ts_fn not in ['0.0']):
            
            print(glac_idx, glac_str)
            
            # Create glacier feature
            gf = create_glacfeat(thick_dir, thick_fn)
        
            # Debris shape layer processing
            dc_shp_proj_fn = (debris_prms.glac_shp_proj_fp + glac_str + '_dc_crs' + 
                              str(gf.aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp')
            if not os.path.exists(dc_shp_proj_fn):
                dc_shp_init = gpd.read_file(debris_prms.debriscover_fp + 
                                            debris_prms.debriscover_fn_dict[debris_prms.roi])
                dc_shp_single = dc_shp_init[dc_shp_init['RGIId'] == rgiid]
                dc_shp_single = dc_shp_single.reset_index()
                dc_shp_proj = dc_shp_single.to_crs({'init': 'epsg:' + 
                                                    str(gf.aea_srs.GetAttrValue("AUTHORITY", 1))})
                dc_shp_proj.to_file(dc_shp_proj_fn)
            dc_shp_ds = ogr.Open(dc_shp_proj_fn, 0)
            dc_shp_lyr = dc_shp_ds.GetLayer()

            # Add layers
            gf.add_layers(dc_shp_lyr, gf_add_ts=True, ts_fn=ts_fn, gf_add_slope_aspect=False)

#             # ===== PLOTS =====
#             if debug:
#                 # DEM
#                 var_full2plot = gf.z1.copy()
#                 clim = malib.calcperc(var_full2plot, (2,98))
#                 plot_array(var_full2plot, clim, [glac_str + ' DEM'], 'inferno', 'elev (masl)', close_fig=False)
#                 # Surface temperature
#                 var_full2plot = gf.ts.copy()
#                 clim = malib.calcperc(var_full2plot, (2,98))
#                 plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'ts (degC)', close_fig=False)
#                 # Surface temperature (debris-covered)
#                 var_full2plot = gf.ts.copy()
#                 var_full2plot.mask = gf.dc_mask
#                 clim = malib.calcperc(var_full2plot, (2,98))
#                 plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'ts (degC)', close_fig=False)
            

            # ===== SURFACE TEMPERATURE FOR THINNEST DEBRIS (connected to terminus) =====
            outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)   
            bin_idx_dc = np.where(outbins_df['dc_bin_count_valid'] > 0)[0]
            bin_idx_dif = list(bin_idx_dc[1:] - bin_idx_dc[:-1])
            if not(np.sum(bin_idx_dif) == len(bin_idx_dc)-1):
                idx_jumpinbins = bin_idx_dif.index(next(filter(lambda x: x>1, bin_idx_dif)))
                bin_idx_dc = bin_idx_dc[0:idx_jumpinbins+1]
            
            ts_min = np.nanmin(outbins_df.loc[bin_idx_dc,'dc_ts_med'].values)
            ts_max = np.nanmax(outbins_df.loc[bin_idx_dc,'dc_ts_med'].values)
            
            if debug:
                print('ts_min:', np.round(ts_min,1), ' ts_max:', np.round(ts_max,1))
            
            if np.isnan(ts_min) and np.isnan(ts_max):
                troubleshoot_fp = (debris_prms.output_fp + 'errors/no_Ts_data-extrap/' + debris_prms.roi + '/')
                if not os.path.exists(troubleshoot_fp):
                    os.makedirs(troubleshoot_fp)
                txt_fn_extrapfail = glac_str + "-noTs-extrap.txt"
                with open(troubleshoot_fp + txt_fn_extrapfail, "w") as text_file:
                    text_file.write(glac_str + ' no surface temperature data but made it past preprocessing to extrap')

            else:
                # ===== ESTIMATE DEBRIS THICKNESS FOR EACH GLACIER INDIVIDUALLY =====
                # Load parameters from nearest neighbor
                rgiid_nearest_list = nearest_dict[main_glac_rgi_missing.loc[glac_idx, 'rgino_str']]

                n_nearest = 0
                n_success = 0
                min_n_nearest = 10
                hd_ts_list = []
                mf_list = []
                while n_nearest < n_glac_nearest and n_success < min_n_nearest:
                    rgi_str_nearest = rgiid_nearest_list[n_nearest]
                    if rgi_str_nearest.startswith('0'):
                        rgi_str_nearest = rgi_str_nearest[1:]
#                     if debug:
#                         print(n_nearest, 'rgi nearest:', rgi_str_nearest)

                    # Load parameters
                    df_opt_fn = rgi_str_nearest + '_hdopt_prms.csv'
                    df_opt = pd.read_csv(hdopt_prms_fp + df_opt_fn)
                    melt_2cm = df_opt.loc[0,'melt_mwea_2cm']
                    melt_cleanice = df_opt.loc[0,'melt_mwea_clean']
                    func_coeff = [df_opt.loc[0,'b0'], df_opt.loc[0,'k']]
                    func_coeff_ts = [df_opt.loc[0,'a'], df_opt.loc[0,'b'], df_opt.loc[0,'c']]

                    # Estimate debris thickness of thinnest bin
                    hd_thin = debris_fromts_func(ts_min, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                    hd_thick = debris_fromts_func(ts_max, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                    
                    if debug:
                        print('     thin:', np.round(ts_min,1), np.round(hd_thin,3), 'm', 
                              '    thick:', np.round(ts_max,1), np.round(hd_thick,3), 'm')

    #                 print(func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2], ts_min, hd_thin)

                    # Minimum and maximum debris thickness are reasonable
                    if hd_thin > 0 and hd_thin < 0.2 and hd_thick < debris_prms.hd_max:

                        if debug:
                            print('  ', n_nearest, 'hd thin:', np.round(hd_thin,2), 'hd thick:', np.round(hd_thick,2))

                        hd_array = debris_fromts_maskedarray(gf.ts, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                        hd_array[hd_array>debris_prms.hd_max] = debris_prms.hd_max
                        hd_array[hd_array<0] = 0

                        hd_ts_list.append(hd_array)
                        n_success += 1

                    n_nearest += 1
                    
                if len(hd_ts_list) == 0:
                    # Record initial failure
                    troubleshoot_fp = (debris_prms.output_fp + 'errors/extrap_failed_rnd1/' + debris_prms.roi + '/')
                    if not os.path.exists(troubleshoot_fp):
                        os.makedirs(troubleshoot_fp)
                    txt_fn_extrapfail = glac_str + "-extrap_failed_rnd1.txt"
                    with open(troubleshoot_fp + txt_fn_extrapfail, "w") as text_file:
                        text_file.write(glac_str + ' failed to find any reasonable extrapolation estimates in first round')
                    
                    
                # ===== SECOND ROUND: NEGATIVE VALUES CAUSE HD_THIN TO BE 0 =====
                #  assume the absolute surface temperature is wrong, but spatial variation is representative
                #  of debris thickness variations, so increase surface temperature until find good fit
                if len(hd_ts_list) == 0 and ts_min < 0:
                    
                    if debug:
                        print('\n-----\nROUND 2 OF EXTRAPOLATION')
                    
                    ts_offset = abs(ts_min)
                    n_offset = 0
                    while len(hd_ts_list) < 5 and n_offset < 20:
                        gf.dc_ts = np.ma.array(gf.dc_ts.data.copy() + ts_offset, mask=gf.dc_ts.mask)
#                             ts_array = gf.ts.data.copy() + ts_offset
                        gf.ts = np.ma.array(gf.ts.data.copy() + ts_offset, mask=gf.ts.mask)
                        outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)
                        ts_min = np.nanmin(outbins_df.loc[bin_idx_dc,'dc_ts_med'].values)
                        ts_max = np.nanmax(outbins_df.loc[bin_idx_dc,'dc_ts_med'].values)
                        # Estimate debris thickness of thinnest bin
                        hd_thin = debris_fromts_func(ts_min, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                        hd_thick = debris_fromts_func(ts_max, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])

                        if debug:
                            print('n_offset:', n_offset, ts_min, ts_max)

                        # ===== ESTIMATE DEBRIS THICKNESS FOR EACH GLACIER INDIVIDUALLY =====
                        # Load parameters from nearest neighbor
                        rgiid_nearest_list = nearest_dict[main_glac_rgi_missing.loc[glac_idx, 'rgino_str']]

                        n_nearest = 0
                        n_success = 0
                        min_n_nearest = 10
                        hd_ts_list = []
                        mf_list = []
                        while n_nearest < n_glac_nearest and n_success < min_n_nearest:
                            rgi_str_nearest = rgiid_nearest_list[n_nearest]
                            if rgi_str_nearest.startswith('0'):
                                rgi_str_nearest = rgi_str_nearest[1:]

                            # Load parameters
                            df_opt_fn = rgi_str_nearest + '_hdopt_prms.csv'
                            df_opt = pd.read_csv(hdopt_prms_fp + df_opt_fn)
                            melt_2cm = df_opt.loc[0,'melt_mwea_2cm']
                            melt_cleanice = df_opt.loc[0,'melt_mwea_clean']
                            func_coeff = [df_opt.loc[0,'b0'], df_opt.loc[0,'k']]
                            func_coeff_ts = [df_opt.loc[0,'a'], df_opt.loc[0,'b'], df_opt.loc[0,'c']]

                            # Estimate debris thickness of thinnest bin
                            hd_thin = debris_fromts_func(ts_min, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                            hd_thick = debris_fromts_func(ts_max, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                            # Minimum and maximum debris thickness are reasonable
                            if hd_thin > 0.01 and hd_thin < 0.2 and hd_thick < debris_prms.hd_max:

                                if debug:
                                    print('  ', n_nearest, 'hd thin:', np.round(hd_thin,2), 'hd thick:', np.round(hd_thick,2))

                                hd_array = debris_fromts_maskedarray(gf.ts, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                                hd_array[hd_array>debris_prms.hd_max] = debris_prms.hd_max
                                hd_array[hd_array<0] = 0

                                hd_ts_list.append(hd_array)
                                n_success += 1

                            n_nearest += 1

                        n_offset += 1
                        ts_offset = 1
                        
                        
                if len(hd_ts_list) == 0:
                    # Record initial failure
                    troubleshoot_fp = (debris_prms.output_fp + 'errors/extrap_failed_rnd2/' + debris_prms.roi + '/')
                    if not os.path.exists(troubleshoot_fp):
                        os.makedirs(troubleshoot_fp)
                    txt_fn_extrapfail = glac_str + "-extrap_failed_rnd2.txt"
                    with open(troubleshoot_fp + txt_fn_extrapfail, "w") as text_file:
                        text_file.write(glac_str + ' failed to find any reasonable extrapolation estimates in second round')
                    
                # ===== THIRD ROUND: ASSUME TOO POSITIVE CAUSING HD TO BE VERY THICK =====
                #  assume the absolute surface temperature is wrong, but spatial variation is representative
                #  of debris thickness variations, so increase surface temperature until find good fit
                if len(hd_ts_list) == 0 and ts_max > 20:
                    
                    if debug:
                        print('\n-----\nROUND 3 OF EXTRAPOLATION')
                    
                    ts_offset = -1
                    n_offset = 0
                    while len(hd_ts_list) < 5 and n_offset < 20:
                        gf.dc_ts = np.ma.array(gf.dc_ts.data.copy() + ts_offset, mask=gf.dc_ts.mask)
                        gf.ts = np.ma.array(gf.ts.data.copy() + ts_offset, mask=gf.ts.mask)
                        outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)
                        ts_min = np.nanmin(outbins_df.loc[bin_idx_dc,'dc_ts_med'].values)
                        ts_max = np.nanmax(outbins_df.loc[bin_idx_dc,'dc_ts_med'].values)
                        # Estimate debris thickness of thinnest bin
                        hd_thin = debris_fromts_func(ts_min, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                        hd_thick = debris_fromts_func(ts_max, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])

                        if debug:
                            print('n_offset:', n_offset, ts_min, ts_max)

                        # ===== ESTIMATE DEBRIS THICKNESS FOR EACH GLACIER INDIVIDUALLY =====
                        # Load parameters from nearest neighbor
                        rgiid_nearest_list = nearest_dict[main_glac_rgi_missing.loc[glac_idx, 'rgino_str']]

                        n_nearest = 0
                        n_success = 0
                        min_n_nearest = 10
                        hd_ts_list = []
                        mf_list = []
                        while n_nearest < n_glac_nearest and n_success < min_n_nearest:
                            rgi_str_nearest = rgiid_nearest_list[n_nearest]
                            if rgi_str_nearest.startswith('0'):
                                rgi_str_nearest = rgi_str_nearest[1:]

                            # Load parameters
                            df_opt_fn = rgi_str_nearest + '_hdopt_prms.csv'
                            df_opt = pd.read_csv(hdopt_prms_fp + df_opt_fn)
                            melt_2cm = df_opt.loc[0,'melt_mwea_2cm']
                            melt_cleanice = df_opt.loc[0,'melt_mwea_clean']
                            func_coeff = [df_opt.loc[0,'b0'], df_opt.loc[0,'k']]
                            func_coeff_ts = [df_opt.loc[0,'a'], df_opt.loc[0,'b'], df_opt.loc[0,'c']]

                            # Estimate debris thickness of thinnest bin
                            hd_thin = debris_fromts_func(ts_min, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                            hd_thick = debris_fromts_func(ts_max, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                            # Minimum and maximum debris thickness are reasonable
                            if hd_thin > 0.01 and hd_thin < 0.2 and hd_thick < debris_prms.hd_max:

                                if debug:
                                    print('  ', n_nearest, 'hd thin:', np.round(hd_thin,2), 'hd thick:', np.round(hd_thick,2))

                                hd_array = debris_fromts_maskedarray(gf.ts, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                                hd_array[hd_array>debris_prms.hd_max] = debris_prms.hd_max
                                hd_array[hd_array<0] = 0

                                hd_ts_list.append(hd_array)
                                n_success += 1

                            n_nearest += 1

                        n_offset += 1

                # ===== ESTIMATE DEBRIS THICKNESS FROM ALL COMBINATIONS =====
                if len(hd_ts_list) > 0:
                    # DEBRIS THICKNESS based on median of the plausible nearest values
                    hd_ts_all = np.array(hd_ts_list)
                    hd_ts_med = np.median(hd_ts_all, axis=0)
                    gf.debris_thick_ts = np.ma.array(hd_ts_med, mask=gf.dc_mask)

                    if debug:
                        close_fig=False
                    else:
                        close_fig=True

                    # Debris thickness
                    var_full2plot = gf.debris_thick_ts.copy()
                    clim = (0,1)
                    plot_array(var_full2plot, clim, [gf.glacnum + ' hd (from ts)'], 'inferno', 'hd (m)', 
                               fn=fig_extrap + gf.feat_fn +'_hd_ts.png', close_fig=close_fig)

                    # ===== EXPORT DEBRIS THICKNESS AND MELT FACTOR TIFS ===== 
                    # Debris thickness
                    gf.debris_thick_ts.mask = gf.dc_mask
                    iolib.writeGTiff(gf.debris_thick_ts, hd_fp + hd_fn, gf.ds_dict['z1'])

                    # Melt factor
                    gf.meltfactor_ts = (
                        melt_fromdebris_func(gf.debris_thick_ts, func_coeff[0], func_coeff[1]) / melt_cleanice)
                    # limit melt rates to modeled 2 cm rate
                    gf.meltfactor_ts = np.ma.array(
                        maskedarray_gt(gf.meltfactor_ts, melt_2cm / melt_cleanice), 
                        mask=np.ma.getmask(gf.debris_thick_ts))
                    # Linearly interpolate between 0 cm and 2 cm for the melt rate
                    def meltfactor_0to2cm_adjustment(mf, melt_clean, melt_2cm, hd):
                        """ Linearly interpolate melt factors between 0 and 2 cm 
                            based on clean ice and 2 cm sub-debris melt """
                        mf = np.nan_to_num(mf,0)
                        mf[(hd >= 0) & (hd < 0.02)] = (
                            1 + hd[(hd >= 0) & (hd < 0.02)] / 0.02 * (melt_2cm - melt_clean) / melt_clean)
                        return mf
                    gf.meltfactor_ts = np.ma.array(
                        meltfactor_0to2cm_adjustment(gf.meltfactor_ts.data.copy(), melt_cleanice, melt_2cm, 
                                                     gf.debris_thick_ts.data), 
                        mask=np.ma.getmask(gf.debris_thick_ts))
                    # Plot melt factor
                    var_full2plot = gf.meltfactor_ts.copy()
                    clim = (0,1.25)
                    plot_array(var_full2plot, clim, [gf.glacnum + ' melt factor'], 'inferno', 'melt factor (-)', 
                               fn=fig_extrap + gf.feat_fn +'_mf.png', close_fig=True)

                    gf.meltfactor_ts.mask = gf.dc_mask
                    mf_fn = debris_prms.mf_fn_sample.replace('XXXX',gf.glacnum).replace('.tif','_extrap.tif')
                    iolib.writeGTiff(gf.meltfactor_ts, mf_fp + mf_fn, gf.ds_dict['z1'])


                    # ===== EXPORT THE BINNED DEBRIS THICKNESS AND MELT FACTOR =====
                    # Output debris thickness
                    outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)
                    hd_extrap_bin_fp = debris_prms.mb_binned_fp_wdebris_hdts + '../_wdebris_hdts_extrap/'
                    if not os.path.exists(hd_extrap_bin_fp):
                        os.makedirs(hd_extrap_bin_fp)
                    outbins_df.to_csv(hd_extrap_bin_fp + glac_str + '_mb_bins_hdts_extrap.csv', index=False)

                else:
                    troubleshoot_fp = (debris_prms.output_fp + 'errors/extrap_failed/' + debris_prms.roi + '/')
                    if not os.path.exists(troubleshoot_fp):
                        os.makedirs(troubleshoot_fp)
                    txt_fn_extrapfail = glac_str + "-extrap_failed.txt"
                    with open(troubleshoot_fp + txt_fn_extrapfail, "w") as text_file:
                        text_file.write(glac_str + ' failed to find any reasonable extrapolation estimates')

0 03.00093
500 03.01328
1000 03.02171
1500 03.03271
2000 03.04077
0 03.00093
1 03.00094
2 03.00095
3 03.00096
4 03.00097


/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/numpy/ma/core.py:4008: RuntimeWarning: invalid value encountered in equal
  check = compare(sdata, odata)


5 03.00098
6 03.00099
7 03.00100
8 03.00102
9 03.00103
10 03.00104
11 03.00105
12 03.00106
13 03.00107
14 03.00108


/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/ipykernel_launcher.py:113: RuntimeWarning: All-NaN slice encountered
/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/ipykernel_launcher.py:114: RuntimeWarning: All-NaN slice encountered


15 03.00109
16 03.00110
17 03.00113
18 03.00114
19 03.00115
20 03.00116
21 03.00117
22 03.00118
23 03.00120
24 03.00123
25 03.00124
26 03.00125
27 03.00126
28 03.00128
29 03.00129
30 03.00130
31 03.00131
32 03.00132
33 03.00133
34 03.00134
35 03.00135
36 03.00136
37 03.00137
38 03.00139
39 03.00140
40 03.00141
41 03.00142
42 03.00143
43 03.00144
44 03.00145
45 03.00146
46 03.00147
47 03.00148
48 03.00150
49 03.00151
50 03.00152
51 03.00153
52 03.00155
53 03.00156
54 03.00157
55 03.00158
56 03.00159
57 03.00161
58 03.00162
59 03.00163
60 03.00167
61 03.00171
62 03.00174
63 03.00175
64 03.00176
65 03.00180
66 03.00182
67 03.00183
68 03.00185
69 03.00186
70 03.00188
71 03.00189
72 03.00191
73 03.00195
74 03.00196
75 03.00197
76 03.00204
77 03.00205
78 03.00207
79 03.00212
80 03.00213
81 03.00217
82 03.00222
83 03.00237
84 03.00240
85 03.00242
86 03.00251
87 03.00254
88 03.00256
89 03.00262
90 03.00264
91 03.00265
92 03.00269
93 03.00271
94 03.00283
95 03.00295
96 03.00296
97 03.00300
98 0

/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in greater
/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in less
/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/ipykernel_launcher.py:169: RuntimeWarning: invalid value encountered in greater
/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/ipykernel_launcher.py:170: RuntimeWarning: invalid value encountered in less
/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/ipykernel_launcher.py:365: RuntimeWarning: invalid value encountered in greater_equal
/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/ipykernel_launcher.py:365: RuntimeWarning: invalid value encountered in less


586 03.01477
587 03.01478
588 03.01480
589 03.01481
590 03.01482
591 03.01483
592 03.01485
593 03.01490
594 03.01493
595 03.01495
596 03.01496
597 03.01497
598 03.01498
599 03.01499
600 03.01501
601 03.01502
602 03.01503
603 03.01504
604 03.01506
605 03.01508
606 03.01509
607 03.01512
608 03.01513
609 03.01514
610 03.01515
611 03.01516
612 03.01517
613 03.01518
614 03.01519
615 03.01520
616 03.01522
617 03.01523
618 03.01525
619 03.01526
620 03.01527
621 03.01528
622 03.01529
623 03.01530
624 03.01531
625 03.01533
626 03.01535
627 03.01536
628 03.01537
629 03.01538
630 03.01539
631 03.01544
632 03.01545
633 03.01546
634 03.01550
635 03.01552
636 03.01553
637 03.01555
638 03.01564
639 03.01565
640 03.01566
641 03.01570
642 03.01571
643 03.01572
644 03.01573
645 03.01575
646 03.01576
647 03.01577
648 03.01579
649 03.01582
650 03.01583
651 03.01585
652 03.01586
653 03.01587
654 03.01588
655 03.01589
656 03.01590
657 03.01593
658 03.01598
659 03.01599
660 03.01600
661 03.01601
662 03.01602

1201 03.02471
1202 03.02472
1203 03.02475
1204 03.02476
1205 03.02480
1206 03.02481
1207 03.02484
1208 03.02488
1209 03.02489
1210 03.02490
1211 03.02491
1212 03.02498
1213 03.02503
1214 03.02506
1215 03.02507
1216 03.02508
1217 03.02514
1218 03.02515
1219 03.02517
1220 03.02518
1221 03.02521
1222 03.02525
1223 03.02527
1224 03.02531
1225 03.02533
1226 03.02535
1227 03.02536
1228 03.02539
1229 03.02540
1230 03.02543
1231 03.02553
1232 03.02554
1233 03.02558
1234 03.02560
1235 03.02563
1236 03.02564
1237 03.02565
1238 03.02568
1239 03.02570
1240 03.02572
1241 03.02573
1242 03.02577
1243 03.02578
1244 03.02582
1245 03.02583
1246 03.02588
1247 03.02595
1248 03.02597
1249 03.02606
1250 03.02607
1251 03.02609
1252 03.02610
1253 03.02611
1254 03.02612
1255 03.02613
1256 03.02615
1257 03.02616
1258 03.02617
1259 03.02618
1260 03.02620
1261 03.02622
1262 03.02625
1263 03.02626
1264 03.02627
1265 03.02628
1266 03.02629
1267 03.02630
1268 03.02631
1269 03.02632
1270 03.02633
1271 03.02634
1272 0

/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/ipykernel_launcher.py:243: RuntimeWarning: invalid value encountered in greater
/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/ipykernel_launcher.py:244: RuntimeWarning: invalid value encountered in less


1701 03.03554
1702 03.03555
1703 03.03556
1704 03.03557
1705 03.03558
1706 03.03559
1707 03.03560
1708 03.03561
1709 03.03562
1710 03.03563
1711 03.03564
1712 03.03565
1713 03.03566
1714 03.03567
1715 03.03568
1716 03.03569
1717 03.03570
1718 03.03571
1719 03.03572
1720 03.03573
1721 03.03574
1722 03.03575
1723 03.03576
1724 03.03577
1725 03.03578
1726 03.03579
1727 03.03580
1728 03.03581
1729 03.03582
1730 03.03583
1731 03.03584
1732 03.03585
1733 03.03586
1734 03.03587
1735 03.03588
1736 03.03589
1737 03.03590
1738 03.03591
1739 03.03592
1740 03.03594
1741 03.03596
1742 03.03597
1743 03.03598
1744 03.03599
1745 03.03600
1746 03.03601
1747 03.03602
1748 03.03603
1749 03.03604
1750 03.03605
1751 03.03606
1752 03.03607
1753 03.03608
1754 03.03609
1755 03.03610
1756 03.03611
1757 03.03612
1758 03.03613
1759 03.03614
1760 03.03615
1761 03.03616
1762 03.03618
1763 03.03619
1764 03.03620
1765 03.03621
1766 03.03622
1767 03.03623
1768 03.03624
1769 03.03625
1770 03.03626
1771 03.03627
1772 0

In [7]:
# bin_idx_dc_thin = bin_idx_dc[-1]
# outbins_df.loc[bin_idx_dc_thin, 'hd_ts_med_m']
# outbins_df.loc[:,['bin_center_elev_m', 'dc_ts_med', 'hd_ts_med_m']]

In [8]:
print('\nDONE!\n')


DONE!



In [9]:
print('TO-DO LIST:')
print('  - Mosaic extrapolated and calibrated glaciers into 1-degree pixels')
print('  - Test robustness on glaciers with data (Ngozumpa, Miage, etc.)')

TO-DO LIST:
  - Mosaic extrapolated and calibrated glaciers into 1-degree pixels
  - Test robustness on glaciers with data (Ngozumpa, Miage, etc.)
